# Plot archived radar imagery

Using the Australian Unified Radar Archive (https://www.openradar.io/operational-network), we can plot an animation of radar data for selected locations and dates, to enable verification of storm events. 

In [ ]:
%matplotlib inline

import os
import tempfile
import zipfile
import urllib
from datetime import datetime
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import pyart
from matplotlib import animation
from IPython.display import HTML
import cartopy.crs as ccrs
import imageio.v2 as imageio


In [ ]:
"""
CONFIG
"""

#Specific the radar and date we want to download
radar_id     = 16 #this is the Marburg radar near Brisbane.
date_str     = '2011/02/21' #in yyyy/mm/dd
base_url     = 'http://dapds00.nci.org.au/thredds/fileServer/rq0' #base url for NCI dataset

#specify start and end times for plotting a subset of radar volumes
start_str = '01:00' #time in UTC
end_str   = '05:00' #time in UTC
#specify radar tilt and field to plot
tilt      = 2 #second tilt!
field     = 'DBZH' #reflectivity

In [ ]:
#parse inputs
date_dt      = datetime.strptime(date_str,'%Y/%m/%d')

#build request filename url
tar_fn       = str(radar_id) + '_' + date_dt.strftime('%Y%m%d') + '.pvol.zip'
request_url  = '/'.join([base_url, str(radar_id), date_dt.strftime('%Y'), 'vol', tar_fn])
print('my request is ',request_url)

In [ ]:
#downloading and extracting the data

#download the zip file
if not os.path.isfile(tar_fn):
    urllib.request.urlretrieve(request_url, tar_fn)

#extract the zip file to a temporary directory
temp_dir = tempfile.mkdtemp()
zip_fh = zipfile.ZipFile(tar_fn)
zip_fh.extractall(path = temp_dir)
zip_fh.close()

#list all the volumes extracted from the zip file
file_list = sorted(glob(temp_dir + '/*'))
print('\n'.join(file_list)) #print with newline between each list item

In [ ]:
#first convert the start/end time strings into datetime numbers
start_dt  = datetime.strptime(date_str + ' ' + start_str, '%Y/%m/%d %H:%M')
end_dt    = datetime.strptime(date_str + ' ' + end_str, '%Y/%m/%d %H:%M')

#now let's read the datetime numbers of all the volumes for comparision
file_dt_list = []
for i, fname in enumerate(file_list):
    file_dt_list.append(datetime.strptime(os.path.basename(fname)[3:18],'%Y%m%d_%H%M%S'))
    
#find the index of volumes within our start and end times
file_dt_array = np.array(file_dt_list)
index_array = np.where(np.logical_and(file_dt_array >= start_dt, file_dt_array<=end_dt))[0]

In [ ]:
#build list of radar objects to plot
radar_list = []
for index in index_array:
    #get file name of index
    file_name = file_list[index]
    #open volume using pyart
    try:
        my_radar = pyart.aux_io.read_odim_h5(file_name, file_field_names=True)
    except:
        print('failed to open file', file_name)
        continue
    #clean up field metadata
    my_radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    my_radar.fields['DBZH']['units'] = 'dBZ'
    my_radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    #append to radar list for animation later
    radar_list += [my_radar]

#determine plot domains
radar_lat = my_radar.latitude['data'][0]
radar_lon = my_radar.longitude['data'][0]
min_lat   = radar_lat - 1.0
max_lat   = radar_lat + 1.0
min_lon   = radar_lon - 1.5
max_lon   = radar_lon + 1.5

In [ ]:
#generate animation of reflectivity

# Set up the GIS projection
projection = ccrs.Mercator(
                central_longitude=radar_lon,
                min_latitude=min_lat, max_latitude=max_lat)

def plot_reflectivity(radarframe, seq):
    fig = plt.figure(figsize=(16, 12))
    display = pyart.graph.RadarMapDisplay(radarframe)
    display.plot_ppi_map('DBZH', tilt,
                         projection=projection, colorbar_flag=False,
                         min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                         vmin=0, vmax=64, cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                         resolution='10m')
    lb = display._get_colorbar_label('DBZH')
    cb = plt.colorbar(display.plots[0], aspect=30, pad=0.07, 
                      orientation='horizontal')
    cb.ax.tick_params(labelsize=10)
    cb.set_label(lb, fontsize=10)

    #Now we add lat lon lines
    gl = display.ax.gridlines(draw_labels=True,
                              linewidth=2, color='gray', alpha=0.5,
                              linestyle='--')
    gl.xlabel_style = {'size': 10}
    gl.ylabel_style = {'size': 10}
    gl.top_labels = False
    gl.right_labels = False
    outputfile = os.path.join(temp_dir, f"radar.{seq:04d}.png")
    plt.savefig(outputfile)
    return imageio.imread(outputfile)

imglist = []
for idx, frame in enumerate(radar_list):
    img = plot_reflectivity(frame, idx)
    imglist.append(img)

dtstr = date_dt.strftime('%Y%m%d')
imageio.mimwrite(f"radar.{radar_id:03d}.{dtstr}.{field}.gif", imglist, fps=5)
